# Genetic Algorithm MLP

In [5]:
import random
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


**1. Dataset Preparation**


In [ ]:
mnist = fetch_openml('mnist_784',version = 1)
X = mnist.data
y = mnist.target.astype(int)
X_filtered = X[(y == 0) | (y == 1)]
y_filtered = y[(y == 0) | (y == 1)]

X_normalized = X_filtered / 255.0
X_train,X_test,y_train,y_test=train_test_split(X_normalized,y_filtered,test_size=0.2,random_state=42)

**2: Define the MLP and Fitness Function**

In [11]:
def evaluate_mlp(hidden_layer_sizes, learning_rate, alpha, X_train, y_train, X_test, y_test):
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        learning_rate_init=learning_rate,
        alpha=alpha,
        max_iter=200,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    error = 1 - accuracy_score(y_test, y_pred)
    return error

def fitness_function(chromosome):
    hidden_layer_size = (int(chromosome[0]),)
    learning_rate = chromosome[1]
    alpha = chromosome[2]
    return evaluate_mlp(hidden_layer_size, learning_rate, alpha, X_train, y_train, X_test, y_test)

**3: Genetic Algorithm Components**

In [12]:
# Initialization
def generate_population(size, n_neurons_range, lr_range, alpha_range):
    population = []
    for _ in range(size):
        chromosome = [
            random.randint(*n_neurons_range),   # Number of neurons
            random.uniform(*lr_range),         # Learning rate
            random.uniform(*alpha_range)       # Regularization (alpha)
        ]
        population.append(chromosome)
    return population

# Selection
def roulette_wheel_selection(population, fitness_scores):
    total_fitness = sum(1 / score for score in fitness_scores)
    probabilities = [(1 / score) / total_fitness for score in fitness_scores]
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]

# Crossover
def crossover(parent1, parent2, alpha=0.7):
    child1 = [
        alpha * p1 + (1 - alpha) * p2
        for p1, p2 in zip(parent1, parent2)
    ]
    child2 = [
        (1 - alpha) * p1 + alpha * p2
        for p1, p2 in zip(parent1, parent2)
    ]
    return child1, child2


# Mutation
def mutate(chromosome, n_neurons_range, lr_range, alpha_range, mutation_rate=0.1):
    if random.random() < mutation_rate:
        gene_to_mutate = random.randint(0, 2)  # Select random gene
        if gene_to_mutate == 0:
            chromosome[0] = random.randint(*n_neurons_range)  # Neurons
        elif gene_to_mutate == 1:
            chromosome[1] = random.uniform(*lr_range)         # Learning rate
        elif gene_to_mutate == 2:
            chromosome[2] = random.uniform(*alpha_range)      # Regularization
    return chromosome

# Replacement
def replace_population(population, fitness_scores, new_children):
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population))]
    return sorted_population[:-len(new_children)] + new_children

**4: Run Genetic Algorithm**

In [13]:
def genetic_algorithm(
    generations, population_size, n_neurons_range, lr_range, alpha_range,
    mutation_rate, X_train, y_train, X_test, y_test
):
    population = generate_population(population_size, n_neurons_range, lr_range, alpha_range)
    for generation in range(generations):
        # Evaluate fitness
        fitness_scores = [fitness_function(ind) for ind in population]

        # Log best fitness
        best_index = np.argmin(fitness_scores)
        best_chromosome = population[best_index]
        print(f"Generation {generation + 1} - Best Fitness: {fitness_scores[best_index]:.4f} - Best Chromosome: {best_chromosome}")

        # Selection
        new_population = []
        for _ in range(population_size // 2):
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])

        # Mutation
        new_population = [
            mutate(ind, n_neurons_range, lr_range, alpha_range, mutation_rate)
            for ind in new_population
        ]

        # Replacement
        population = replace_population(population, fitness_scores, new_population)
    
    # Final evaluation
    fitness_scores = [fitness_function(ind) for ind in population]
    best_index = np.argmin(fitness_scores)
    best_chromosome = population[best_index]
    print(f"Best Chromosome After {generations} Generations: {best_chromosome}")
    return best_chromosome

In [14]:
# Run the Genetic Algorithm
best_solution = genetic_algorithm(
    generations=20,
    population_size=10,
    n_neurons_range=(5, 50),
    lr_range=(0.001, 0.1),
    alpha_range=(0.0001, 0.01),
    mutation_rate=0.1,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test
)
print("Optimal Hyperparameters Found:", best_solution)


Generation 1 - Best Fitness: 0.0003 - Best Chromosome: [37, 0.0370909584163922, 0.007550518956277305]
Generation 2 - Best Fitness: 0.0000 - Best Chromosome: [25.8, 0.06544559631489436, 0.007292923351987359]


ZeroDivisionError: float division by zero